##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 TFX 流水线和 TensorFlow Data Validation 进行数据验证

注：我们建议在 Colab 笔记本中运行本教程，无需进行设置！只需点击“在 Google Colab 中运行”。

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_tfdv"> <img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/penguin_tfdv.ipynb"> <img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/penguin_tfdv.ipynb"> <img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/penguin_tfdv.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table></div>

在这个基于笔记本的教程中，我们将创建和运行 TFX 流水线来验证输入数据并创建机器学习模型。此笔记本基于我们在[简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple)中构建的 TFX 流水线。如果您尚未阅读该教程，请先阅读，然后再继续此笔记本。

所有数据科学或机器学习项目的第一项任务都是理解和清理数据，其中包括：

- 理解每个特征的数据类型、分布和其他信息（例如，平均值或唯一值的个数）
- 生成描述数据的初步架构
- 针对给定架构识别数据中的异常和缺失值

在本教程中，我们将创建两个 TFX 流水线。

首先，我们将创建一个流水线来分析数据集并生成给定数据集的初步架构。此流水线将包含两个新组件：`StatisticsGen` 和 `SchemaGen`。

拥有适当的数据架构后，我们将创建一个流水线以基于上一个教程中的流水线来训练机器学习分类模型。在此流水线中，我们将使用第一个流水线中的架构和一个新的组件 `ExampleValidator` 来验证输入数据。

StatisticsGen、SchemaGen 和 ExampleValidator 这三个新组件是用于数据分析和验证的 TFX 组件，它们是使用 [TensorFlow Data Validation](https://tensorflow.google.cn/tfx/guide/tfdv) 库实现的。

要详细了解 TFX 中的各种概念，请参阅[了解 TFX 流水线](https://tensorflow.google.cn/tfx/guide/understanding_tfx_pipelines)。

## 安装

我们首先需要安装 TFX Python 软件包并下载将用于模型的数据集。

### 升级 Pip

为了避免在本地运行时升级系统中的 Pip，请检查以确保在 Colab 中运行。当然，可以对本地系统单独升级。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### 安装 TFX


In [ ]:
!pip install -U tfx

### 卸载 Shapely

TODO(b/263441833) 这是避免 ImportError 的临时解决方案。最终，应该通过支持最新版本的 Bigquery 来处理，而不是卸载其他额外的依赖项。

In [ ]:
!pip uninstall shapely -y

### 是否已重新启动运行时？

如果您使用的是 Google Colab，则在首次运行上面的代码单元时必须重新启动运行时，方法是点击上面的“RESTART RUNTIME”按钮或使用“Runtime &gt; Restart runtime ...”菜单。这样做的原因是 Colab 加载软件包的方式。

检查 TensorFlow 和 TFX 版本。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 设置变量

有一些变量用于定义流水线。您可以根据需要自定义这些变量。默认情况下，流水线的所有输出都将在当前目录下生成。

In [ ]:
import os

# We will create two pipelines. One for schema generation and one for training.
SCHEMA_PIPELINE_NAME = "penguin-tfdv-schema"
PIPELINE_NAME = "penguin-tfdv"

# Output directory to store artifacts generated from the pipeline.
SCHEMA_PIPELINE_ROOT = os.path.join('pipelines', SCHEMA_PIPELINE_NAME)
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
SCHEMA_METADATA_PATH = os.path.join('metadata', SCHEMA_PIPELINE_NAME,
                                    'metadata.db')
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### 准备样本数据

我们将下载要在我们的 TFX 流水线中使用的样本数据集。我们要使用的数据集为 [Palmer Penguins 数据集](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)，该数据集也在其他 [TFX 示例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin)中使用。

该数据集中有四个数字特征：

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

所有特征均已归一化为具有范围 [0,1]。我们将建立一个预测企鹅 `species` 的分类模型。

由于 TFX ExampleGen 组件从目录中读取输入，我们需要创建一个目录并将数据集复制到其中。

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

快速浏览一下 CSV 文件。

In [ ]:
!head {_data_filepath}

您应该能够看到五个特征列。`species` 为 0、1 或 2 之一，所有其他特征的值应介于 0 和 1 之间。我们将创建一个 TFX 流水线来分析此数据集。

## 生成初步架构

TFX 流水线是使用 Python API 定义的。我们将创建一个流水线以自动从输入样本生成架构。该架构可由人工审核并根据需要进行调整。架构生成后，就可用于后续任务中的训练和样本验证了。

除了在[简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple)中使用的 `CsvExampleGen` 之外，我们还将使用 `StatisticsGen` 和 `SchemaGen`：

- [StatisticsGen](https://tensorflow.google.cn/tfx/guide/statsgen) 用于计算数据集的统计信息。
- [SchemaGen](https://tensorflow.google.cn/tfx/guide/schemagen) 用于检查统计信息并创建初始数据架构。

请参阅每个组件的指南或 [TFX 组件教程](https://tensorflow.google.cn/tfx/tutorials/tfx/components_keras)以详细了解这些组件。

### 编写流水线定义

我们定义一个函数来创建 TFX 流水线。`Pipeline` 对象表示 TFX 流水线，可以使用 TFX 支持的流水线编排系统之一来运行。

In [ ]:
def _create_schema_pipeline(pipeline_name: str,
                            pipeline_root: str,
                            data_root: str,
                            metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline for schema generation."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Computes statistics over data for visualization and schema generation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Generates schema based on the generated statistics.
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  components = [
      example_gen,
      statistics_gen,
      schema_gen,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### 运行流水线

我们将像上一教程一样使用 `LocalDagRunner`。

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_schema_pipeline(
      pipeline_name=SCHEMA_PIPELINE_NAME,
      pipeline_root=SCHEMA_PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=SCHEMA_METADATA_PATH))

如果流水线成功完成，您应该看到“INFO:absl:Component SchemaGen is finished.”。

我们将检查流水线的输出，以便了解我们的数据集。

### 检查流水线的输出

如上一教程所述，TFX 流水线会生成两种输出：工件和[元数据 DB(MLMD)](https://tensorflow.google.cn/tfx/guide/mlmd)，其中包含工件和流水线执行的元数据。我们在上方单元中定义了这些输出的位置。默认情况下，工件存储在 `pipelines` 目录下，元数据以 sqlite 数据库形式存储在 `metadata` 目录下。

您可以使用 MLMD API 以编程方式定位这些输出。首先，我们将定义一些效用函数来搜索刚刚生成的输出工件。


In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_output_artifacts(metadata, latest_execution.id)

# Non-public APIs, just for showcase.
from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

现在，我们可以检查流水线执行的输出。

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    SCHEMA_METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  stat_gen_output = get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME,
                                         'StatisticsGen')
  stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

  schema_gen_output = get_latest_artifacts(metadata_handler,
                                           SCHEMA_PIPELINE_NAME, 'SchemaGen')
  schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]

是时候检查每个组件的输出了。如上所述，在 `StatisticsGen` 和 `SchemaGen` 中使用了 [Tensorflow Data Validation(TFDV)](https://tensorflow.google.cn/tfx/data_validation/get_started)，并且 TFDV 还提供了这些组件输出的呈现。

在本教程中，我们将使用 TFX 中的呈现辅助方法，它们会在内部使用 TFDV 来显示呈现效果。

#### 检查 StatisticsGen 的输出


In [ ]:
# docs-infra: no-execute
visualize_artifacts(stats_artifacts)

<!-- <img class="tfo-display-only-on-site"
src="images/penguin_tfdv/penguin_tfdv_statistics.png"/> -->

您可以查看输入数据的各种统计信息。这些统计信息会提供给 `SchemaGen` 以自动构造数据初始架构。


#### 检查 SchemaGen 的输出


In [ ]:
visualize_artifacts(schema_artifacts)

此架构由 StatisticsGen 的输出自动推断而来。您应该能够看到 4 个浮点数特征和 1 个整数特征。

### 导出架构以供将来使用

我们需要检查和细化生成的架构。架构经检查后，需要持续用于后续的机器学习模型训练流水线中。换言之，您可能需要将架构文件添加到您的版本控制系统以便用于实际用例。在本教程中，为了简单起见，我们仅将架构复制到预定义的文件系统路径。


In [ ]:
import shutil

_schema_filename = 'schema.pbtxt'
SCHEMA_PATH = 'schema'

os.makedirs(SCHEMA_PATH, exist_ok=True)
_generated_path = os.path.join(schema_artifacts[0].uri, _schema_filename)

# Copy the 'schema.pbtxt' file from the artifact uri to a predefined path.
shutil.copy(_generated_path, SCHEMA_PATH)

架构文件使用了[协议缓冲区文本格式](https://googleapis.dev/python/protobuf/latest/google/protobuf/text_format.html)和 [TensorFlow Metadata 架构 proto](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto) 的实例。

In [ ]:
print(f'Schema at {SCHEMA_PATH}-----')
!cat {SCHEMA_PATH}/*

您应确保检查架构定义，并可能需要视情况对其进行编辑。在本教程中，我们将仅使用未经更改的生成架构。


## 验证输入样本并训练机器学习模型

我们将回到我们在[简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple)中创建的流水线，以训练机器学习模型并使用生成的架构来编写模型训练代码。

我们还将添加 [ExampleValidator](https://tensorflow.google.cn/tfx/guide/exampleval) 组件，该组件将在传入的数据集中查找与架构相关的异常和缺失值。


### 编写模型训练代码

我们需要像在[简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple)中那样编写模型代码。

模型本身与上一教程中的模型相同，但本次我们将使用生成自上一流水线的架构，而非手动指定特征。大部分代码均未改变。唯一的区别是我们不需要在此文件中指定特征的名称和类型。相反，我们将从*架构*文件中读取它们。

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

# We don't need to specify _FEATURE_KEYS and _FEATURE_SPEC any more.
# Those information can be read from the given schema file.

_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model(schema: schema_pb2.Schema) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://tensorflow.google.cn/guide/keras/overview for all API options.

  # ++ Changed code: Uses all features in the schema except the label.
  feature_keys = [f.name for f in schema.feature if f.name != _LABEL_KEY]
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in feature_keys]
  # ++ End of the changed code.

  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # ++ Changed code: Reads in schema file passed to the Trainer component.
  schema = tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema_pb2.Schema())
  # ++ End of the changed code.

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model(schema)
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

现在，您已经完成了为模型训练构建 TFX 流水线的所有准备步骤。

### 编写流水线定义

我们将添加两个新组件：`Importer` 和 `ExampleValidator`。Importer 会将外部文件引入 TFX 流水线。在本例中，它是一个包含架构定义的文件。ExampleValidator 将检查输入数据并验证是否所有输入数据均符合我们提供的数据架构。


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline using predefined schema with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # NEW: Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],  # Pass the imported schema.
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,

      # NEW: Following three components were added to the pipeline.
      statistics_gen,
      schema_importer,
      example_validator,

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### 运行流水线


In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

如果流水线成功完成，您应该看到“INFO:absl:Component Pusher is finished.”。

### 检查流水线的输出

我们已经训练了企鹅分类模型，并且还在 ExampleValidator 组件中验证了输入样本。我们可以像之前的流水线一样分析 ExampleValidator 的输出。

In [ ]:
metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  ev_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                   'ExampleValidator')
  anomalies_artifacts = ev_output[standard_component_specs.ANOMALIES_KEY]

来自 ExampleValidator 的 ExampleAnomalies 也可以得到呈现。

In [ ]:
visualize_artifacts(anomalies_artifacts)

对于每个样本拆分，您都应该看到“No anomalies found”。由于我们使用了在此流水线中生成架构所使用的相同数据，预计不会出现异常。如果您使用新传入数据重复运行此流水线，则 ExampleValidator 应能发现新数据与现有架构之间的任何差异。

如果发现任何异常，您可以检查您的数据以确认是否有任何样本不符合您的假设。其他组件（如 StatisticsGen）的输出可能会非常有用。但是，发现的任何异常都不会阻止流水线的进一步执行。

## 后续步骤

您可以在 https://tensorflow.google.cn/tfx/tutorials 上找到更多资源。

要详细了解 TFX 中的各种概念，请参阅[了解 TFX 流水线](https://tensorflow.google.cn/tfx/guide/understanding_tfx_pipelines)。
